# 2020 Önálló kutatási feladat

In [2]:
import pandas as pd

## Stock data preprocessing

In [36]:
stock = pd.read_csv('..\Adatok\stock-data\SP500.csv')

stock = stock.drop(columns=['High','Low','Adj Close','Volume'])

def Change(row):
    return (row['Close']-row['Open'])
stock['Change'] = stock.apply (lambda row: Change(row), axis=1)

def label(row, threshold): 
    if abs(row['Change']) > threshold:
        if row['Change'] > 0: return int(1)
        else: return int(0)


def generate_data(sensitivity):
    stock['Label'] = stock.apply (lambda row: label(row, sensitivity), axis=1)
    stock_ready = stock.drop(columns=['Open', 'Close', 'Change'])
    stock_ready = stock_ready[stock_ready.Label.notna()]
    return stock_ready 

stock


,Date,Open,Close,Change
0,2000-01-03,1469.250000,1455.219971,-14.030029
1,2000-01-04,1455.219971,1399.420044,-55.799927
2,2000-01-05,1399.420044,1402.109985,2.689941
3,2000-01-06,1402.109985,1403.449951,1.339966
4,2000-01-07,1403.449951,1441.469971,38.020020
...,...,...,...,...
5164,2020-07-14,3141.110107,3197.520020,56.409913
5165,2020-07-15,3225.979980,3226.560059,0.580079
5166,2020-07-16,3208.360107,3215.570068,7.209961
5167,2020-07-17,3224.209961,3224.729980,0.520019


### Itt tudjuk a *generate_data* függvény paraméterével állítani, hogy milyen mértékű változást veszünk figyelembe.

In [37]:
generate_data(20)

,Date,Label
1,2000-01-04,0.0
4,2000-01-07,1.0
14,2000-01-24,0.0
18,2000-01-28,0.0
19,2000-01-31,1.0
...,...,...
5161,2020-07-09,0.0
5162,2020-07-10,1.0
5163,2020-07-13,0.0
5164,2020-07-14,1.0


In [25]:
tweets_dem = pd.read_csv('..\Adatok\obama\obama-tweets.csv')
tweets_dem = tweets_dem.drop(columns=['Username', 'Tweet Link', 'Retweets', 'Likes', 'TweetImageUrl', 'Image'])
tweets_dem

,Date,Tweet-text
0,2019/04/10_17:08,From a big NBA fan congrats to future Hall of ...
1,2019/04/06_06:16,In just a few minutes I’m taking the stage at ...
2,2019/04/05_09:25,A voice everybody should hear.https://twitter....
3,2019/04/04_12:23,Here’s a story about people doing good that's ...
4,2019/04/03_12:07,Great to see Chicago’s historic mayoral race b...
...,...,...
6846,2012/11/06_05:12,25 reasons that 25 people are voting for Presi...
6847,2012/11/06_04:49,At the final rally of his final campaign last ...
6848,2012/11/06_04:00,It’s Election Day! This is your last chance to...
6849,2012/11/06_03:28,Election Day is here! Confirm your polling pla...
